In [1]:
!ls

datalab


In [4]:
%%writefile getfile.py
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


if __name__ == "__main__":
    import sys
    if len(sys.argv) is not 3:
        print "Usage: python google_drive.py drive_file_id destination_file_path"
    else:
        # TAKE ID FROM SHAREABLE LINK
        file_id = sys.argv[1]
        # DESTINATION FILE ON YOUR DISK
        destination = sys.argv[2]
        download_file_from_google_drive(file_id, destination)

Writing getfile.py


In [0]:
!python getfile.py 1oPImizaSFyyTxqofGdDrX0jUJxNkXYWV /content/Test.tar.gz

In [27]:
!ls -l

total 1334952
-rw-r--r-- 1 root root 455984080 Jun 25 15:14 398.36-desktop-win8-win7-64bit-international-whql.exe
drwxr-xr-x 1 root root      4096 Jul 16 18:50 datalab
drwxr-xr-x 2 root root      4096 Jul 16 18:56 nvidia
-rw-r--r-- 1 root root 455491036 Jul 16 19:02 nvidiaL2.zip
-rw-r--r-- 1 root root 455490866 Jul 16 18:58 nvidia.zip


In [26]:
!zip -r nvidiaL2.zip nvidia.zip 

  adding: nvidia.zip (stored 0%)


In [29]:
from google.colab import files

files.download('nvidiaL2.zip')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 33430, 0, 0)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 654, in __init__
    self.finish()
  File "/usr/lib/python2.7/SocketServer.py", line 713, in finish
    self.wfile.close()
  File "/usr/lib/python2.7/socket.py", line 283, in close
    self.flush()
  File "/usr/lib/python2.7/socket.py", line 307, in flush
    self._sock.sendall(view[write_offset:write_offset+buffer_size])
error: [Errno 32] Broken pipe


In [6]:
!wget https://us.download.nvidia.com/Windows/398.36/398.36-desktop-win8-win7-64bit-international-whql.exe 

--2018-07-16 18:44:06--  https://us.download.nvidia.com/Windows/398.36/398.36-desktop-win8-win7-64bit-international-whql.exe
Resolving us.download.nvidia.com (us.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connecting to us.download.nvidia.com (us.download.nvidia.com)|192.229.211.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455984080 (435M) [application/octet-stream]
Saving to: ‘398.36-desktop-win8-win7-64bit-international-whql.exe’

398.36-desktop-win8 100%[===================>] 434.86M   152MB/s    in 2.9s    

2018-07-16 18:44:09 (152 MB/s) - ‘398.36-desktop-win8-win7-64bit-international-whql.exe’ saved [455984080/455984080]



In [25]:
!ls -l

total 890136
-rw-r--r-- 1 root root 455984080 Jun 25 15:14 398.36-desktop-win8-win7-64bit-international-whql.exe
drwxr-xr-x 1 root root      4096 Jul 16 18:50 datalab
drwxr-xr-x 2 root root      4096 Jul 16 18:56 nvidia
-rw-r--r-- 1 root root 455490866 Jul 16 18:58 nvidia.zip


In [28]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile()
uploaded.SetContentFile('nvidiaL2.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))
print('title: %s, mimeType: %s' % (uploaded['title'], uploaded['mimeType']))

Uploaded file with ID 1DeteYfuye9pMAXm83imVzaQ3Uk8rRTOp
title: nvidiaL2.zip, mimeType: application/zip
